In [ ]:
from textblob import TextBlob
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition,ensemble

import pandas, xgboost, numpy, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers

In [ ]:
import pandas as pd
data=pd.read_csv("train.tsv",sep="\t")

In [3]:
data.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2


In [4]:
data["Sentiment"].replace(0,value="negative",inplace=True)
data["Sentiment"].replace(1,value="negative",inplace=True)

In [5]:
data["Sentiment"].replace(3,value="positive",inplace=True)
data["Sentiment"].replace(4,value="positive",inplace=True)

In [6]:
data.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,negative
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2


In [7]:
data=data[(data.Sentiment=="negative")|(data.Sentiment=="positive")]

In [8]:
data.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,negative
21,22,1,good for the goose,positive
22,23,1,good,positive
33,34,1,"the gander , some of which occasionally amuses...",negative
46,47,1,amuses,positive


In [9]:
data.groupby("Sentiment").count()

,PhraseId,SentenceId,Phrase
Sentiment,,,
negative,34345,34345,34345
positive,42133,42133,42133


In [10]:
df=pd.DataFrame()
df["text"]=data["Phrase"]
df["label"]=data["Sentiment"]
df.head()

,text,label
0,A series of escapades demonstrating the adage ...,negative
21,good for the goose,positive
22,good,positive
33,"the gander , some of which occasionally amuses...",negative
46,amuses,positive


Text Preprocessing

In [11]:
df["text"]=df["text"].apply(lambda x:  " ".join(x.lower() for x in x.split()))
df["text"]=df["text"].str.replace('[^\w\s]','')
df["text"]=df["text"].str.replace('\d','')
import nltk
from nltk.corpus import stopwords
sw=stopwords.words('english')
df['text']=df['text'].apply(lambda x: " ".join(x for x in x.split() if x not in sw))
delete=pd.Series(' '.join(df['text']).split()).value_counts()[-1000:]
df["text"]=df["text"].apply(lambda x: " ".join(x for x in x.split() if x not in delete))
from textblob import Word
df["text"]=df["text"].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))

Feature Engineering

1-“COUNT VECTORS” 
2-“TF - IDF VECTORS”
-Kendi içinde 3 şekilde uygulanabilir. Bunlar; Word Vectorizer, N-Gram Level ve Characters Level’dır.
3-“WORD EMBEDDING”

In [ ]:
Test- Train

In [13]:
train_x, test_x, train_y, test_y=model_selection.train_test_split(df["text"],df["label"])

In [14]:
encoder=preprocessing.LabelEncoder()

In [15]:
train_y=encoder.fit_transform(train_y)
test_y=encoder.fit_transform(test_y)

In [16]:
train_y[0:5]

array([1, 0, 0, 0, 1])

In [17]:
test_y[0:5]

array([1, 0, 1, 1, 0])

In [ ]:
“COUNT VECTORS” 

In [18]:
vectorizer=CountVectorizer()
vectorizer.fit(train_x)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [19]:
x_train_count=vectorizer.transform(train_x)
x_test_count=vectorizer.transform(test_x)

In [20]:
vectorizer.get_feature_names()[0:5]

['aaa', 'aaliyah', 'abagnale', 'abandon', 'abandoned']

In [21]:
x_train_count.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [ ]:
“TF - IDF VECTORS”

In [22]:
tf_idf_word_vectorizer=TfidfVectorizer()
tf_idf_word_vectorizer.fit(train_x)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [23]:
x_train_tf_idf_word=tf_idf_word_vectorizer.transform(train_x)
x_test_tf_idf_word=tf_idf_word_vectorizer.transform(test_x)

In [24]:
tf_idf_word_vectorizer.get_feature_names()[0:5]

['aaa', 'aaliyah', 'abagnale', 'abandon', 'abandoned']

In [25]:
x_train_tf_idf_word.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
"N-Gram Level TF-IDF"

In [26]:
tf_idf_ngram_vectorizer=TfidfVectorizer(ngram_range=(2,3))
tf_idf_ngram_vectorizer.fit(train_x)


TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(2, 3), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [27]:
x_train_tf_idf_ngram=tf_idf_ngram_vectorizer.transform(train_x)
x_test_tf_idf_ngram=tf_idf_ngram_vectorizer.transform(test_x)

In [ ]:
"Characters Level TF - IDF"

In [28]:
tf_idf_chars_vectorizer=TfidfVectorizer(analyzer="char", ngram_range=(2,3))
tf_idf_chars_vectorizer.fit(train_x)

TfidfVectorizer(analyzer='char', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(2, 3), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [29]:
x_train_tf_idf_chars=tf_idf_chars_vectorizer.transform(train_x)
x_test_tf_idf_chars=tf_idf_chars_vectorizer.transform(test_x)

In [ ]:
LOJISTIK REGRESYON

In [51]:
loj=linear_model.LogisticRegression()
loj_model=loj.fit(x_train_count, train_y)
accuracy=model_selection.cross_val_score(loj_model, x_test_count,test_y, cv=10).mean()
print("count vectors accuracy:", accuracy)

C:\Users\User\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


count vectors accuracy: 0.8368723849372385


In [31]:
loj=linear_model.LogisticRegression()
loj_model=loj.fit(x_train_tf_idf_word, train_y)
accuracy=model_selection.cross_val_score(loj_model, x_test_tf_idf_word,test_y, cv=10).mean()
print("word level tf-idf accuracy:", accuracy)

C:\Users\User\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


word level tf-idf accuracy: 0.8296548117154812


In [32]:
loj=linear_model.LogisticRegression()
loj_model=loj.fit(x_train_tf_idf_ngram, train_y)
accuracy=model_selection.cross_val_score(loj_model, x_test_tf_idf_ngram,test_y, cv=10).mean()
print("n-gram tf-idf accuracy:", accuracy)

n-gram tf-idf accuracy: 0.7446652719665272


In [33]:
loj=linear_model.LogisticRegression()
loj_model=loj.fit(x_train_tf_idf_chars, train_y)
accuracy=model_selection.cross_val_score(loj_model, x_test_tf_idf_chars,test_y, cv=10).mean()
print("char level accuracy:", accuracy)

C:\Users\User\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
C:\Users\User\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regre

char level accuracy: 0.7790271966527196


In [52]:
loj_model

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [36]:
loj_model.predict("yes I like this film")

ValueError: Expected 2D array, got scalar array instead:
array=yes I like this film.
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [53]:
new_=pd.Series('this film is very nice and good i like it')

In [59]:
v=CountVectorizer()
v.fit(train_x)
new_2=v.transform(new_2)

In [55]:
loj_model.predict(new_)

array([1])

In [56]:
new_2=pd.Series('this film is very bad not good')

In [60]:
loj_model.predict(new_2)

array([0])